# Assigment: PyTorch Model Zoo

The [PyTorch "Model Zoo"](https://pytorch.org/vision/stable/models.html) provides a large number of pre-trained CNN models and vision [data sets](https://pytorch.org/vision/stable/datasets.html)...

In [1]:
#imports
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
#transform input data (image) to tensor
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#set batch size
batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170M/170M [00:13<00:00, 12.4MB/s]


## Assignment 1:
Load a "*ResNet18*" from the torchvision model zoo and train it for 10 epochs

In [3]:
# prompt: Load a RestNet18 from the torchvision model zoo and train it for 10 epochs

import torch.optim as optim
import torch.nn as nn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = torchvision.models.resnet18(pretrained=True)
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 10)
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 199MB/s]


[1,  2000] loss: 2.706
[1,  4000] loss: 2.487
[1,  6000] loss: 2.484
[1,  8000] loss: 2.401
[1, 10000] loss: 2.356
[1, 12000] loss: 2.258
[2,  2000] loss: 2.343
[2,  4000] loss: 2.255
[2,  6000] loss: 2.261
[2,  8000] loss: 2.206
[2, 10000] loss: 2.101
[2, 12000] loss: 2.090
[3,  2000] loss: 1.971
[3,  4000] loss: 1.983
[3,  6000] loss: 1.902
[3,  8000] loss: 2.000
[3, 10000] loss: 1.856
[3, 12000] loss: 1.795
[4,  2000] loss: 2.004
[4,  4000] loss: 1.808
[4,  6000] loss: 1.842
[4,  8000] loss: 1.752
[4, 10000] loss: 1.814
[4, 12000] loss: 1.842
[5,  2000] loss: 1.762
[5,  4000] loss: 1.760
[5,  6000] loss: 1.692
[5,  8000] loss: 1.646
[5, 10000] loss: 1.644
[5, 12000] loss: 1.597
[6,  2000] loss: 1.574
[6,  4000] loss: 1.536
[6,  6000] loss: 1.586
[6,  8000] loss: 1.501
[6, 10000] loss: 1.538
[6, 12000] loss: 1.520
[7,  2000] loss: 1.530
[7,  4000] loss: 1.659
[7,  6000] loss: 1.686
[7,  8000] loss: 1.670
[7, 10000] loss: 1.599
[7, 12000] loss: 1.569
[8,  2000] loss: 1.484
[8,  4000] 

In [4]:
# prompt: test the model i want the accuarcy

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the 10000 test images: 36 %


## Assigment 2:
Load a **pre-trained** (on ImageNet) "*ResNet18*" from the torchvision model zoo and *fine-tune* it for ten epochs

In [5]:


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load pre-trained ResNet18
net = torchvision.models.resnet18(pretrained=True)

# Modify the final fully connected layer for CIFAR-10 (10 classes)
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 10)
net.to(device)


# Freeze all layers except the final fully connected layer
for param in net.parameters():
    param.requires_grad = False

for param in net.fc.parameters():
    param.requires_grad = True
for name, param in net.named_parameters():
    if "layer3" in name or "layer4" in name or "fc" in name: # Adjust layer names as needed
        param.requires_grad = True
# Define loss function and optimizer (optimize only the final layer)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, net.parameters()), lr=0.001, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) #Adjust the step_size and gamma values as per your data

for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        scheduler.step()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')




/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[1,  2000] loss: 2.543
[1,  4000] loss: 2.579
[1,  6000] loss: 2.621
[1,  8000] loss: 2.600
[1, 10000] loss: 2.660
[1, 12000] loss: 2.624
[2,  2000] loss: 2.606
[2,  4000] loss: 2.658
[2,  6000] loss: 2.628
[2,  8000] loss: 2.617
[2, 10000] loss: 2.612
[2, 12000] loss: 2.646
[3,  2000] loss: 2.619
[3,  4000] loss: 2.634
[3,  6000] loss: 2.630
[3,  8000] loss: 2.677
[3, 10000] loss: 2.596
[3, 12000] loss: 2.616
[4,  2000] loss: 2.684
[4,  4000] loss: 2.608
[4,  6000] loss: 2.617
[4,  8000] loss: 2.615
[4, 10000] loss: 2.626
[4, 12000] loss: 2.590
[5,  2000] loss: 2.650
[5,  4000] loss: 2.590
[5,  6000] loss: 2.625
[5,  8000] loss: 2.652
[5, 10000] loss: 2.615
[5, 12000] loss: 2.610
[6,  2000] loss: 2.620
[6,  4000] loss: 2.633
[6,  6000] loss: 2.621
[6,  8000] loss: 2.609
[6, 10000] loss: 2.633
[6, 12000] loss: 2.652
[7,  2000] loss: 2.606
[7,  4000] loss: 2.581
[7,  6000] loss: 2.603
[7,  8000] loss: 2.635
[7, 10000] loss: 2.670
[7, 12000] loss: 2.598
[8,  2000] loss: 2.588
[8,  4000] 

KeyboardInterrupt: 